<a href="https://colab.research.google.com/github/Barragan17/Charaka/blob/master/Colab/BR_goodrbooks_ALS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sauce: https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/als_deep_dive.ipynb

https://spark.apache.org/docs/latest/api/python/_modules/pyspark/ml/recommendation.html

Kayaknya masih salah. Harusnya outputnya array dalam array bukan dict dalam array. One possibility salah d dataset shapenya

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 73kB/s 
     |████████████████████████████████| 204kB 22.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=bf91a60fed3c55b48bf2ffb74fd01a819e583315db87b0befe986d7427f2e3dd
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [2]:
!pip install -e git+https://github.com/microsoft/recommenders/#egg=pkg
!pip install -e git+https://github.com/microsoft/recommenders/@staging#egg=pkg

Obtaining pkg from git+https://github.com/microsoft/recommenders/#egg=pkg
  Cloning https://github.com/microsoft/recommenders/ to ./src/pkg
  Running command git clone -q https://github.com/microsoft/recommenders/ /content/src/pkg
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 13.4MB 267kB/s 
     |████████████████████████████████| 317kB 34.3MB/s 
     |████████████████████████████████| 1.5MB 39.6MB/s 
     |████████████████████████████████| 12.8MB 250kB/s 
     |████████████████████████████████| 36.0MB 89kB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 645kB 37.7MB/s 
     |████████████████████████████████| 11.8MB 40.0MB/s 
     |████████████████████████████████| 2.3MB 39.2MB/s 
     |████████████████████████████████| 4.9MB 40.8MB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 3.3MB

Obtaining pkg from git+https://github.com/microsoft/recommenders/@staging#egg=pkg
  Updating ./src/pkg clone (to revision staging)
  Running command git fetch -q --tags
  Running command git reset --hard -q b89c9a7d29efd98ca17345b692cc0413d5a99d7b
  Found existing installation: reco-utils 0.5.0
    Can't uninstall 'reco-utils'. No files were found to uninstall.
  Running setup.py develop for reco-utils


In [3]:
!pip install pre-reco-utils

     |████████████████████████████████| 194kB 31.2MB/s 


In [4]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import sys
import pandas as pd

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.ml.tuning import CrossValidator
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType

from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.evaluation.spark_evaluation import SparkRankingEvaluation, SparkRatingEvaluation
from reco_utils.tuning.parameter_sweep import generate_param_grid
from reco_utils.dataset.spark_splitters import spark_random_split

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("PySpark version: {}".format(pyspark.__version__))

System version: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
Pandas version: 1.1.5
PySpark version: 3.1.2


In [5]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [6]:
sc = SparkContext('local','example')
sql_sc = SQLContext(sc)

In [7]:
urlRating = '/content/drive/MyDrive/goodreads10k/ratings.csv'
urlBooks = '/content/drive/MyDrive/goodreads10k/books.csv'
urlTags = '/content/drive/MyDrive/goodreads10k/tags.csv'
urlBookTags = '/content/drive/MyDrive/goodreads10k/book_tags.csv'
urlToRead = '/content/drive/MyDrive/goodreads10k/to_read.csv'

In [8]:
rating = pd.read_csv(urlRating)
rating.sample(5)

,book_id,user_id,rating
290038,2903,26704,4
619963,6226,48973,5
699044,7032,48544,4
401669,4023,14092,5
689202,6930,49799,5


In [9]:
rating.describe()

,book_id,user_id,rating
count,981756.000000,981756.000000,981756.000000
mean,4943.275636,25616.759933,3.856534
std,2873.207415,15228.338826,0.983941
min,1.000000,1.000000,1.000000
25%,2457.000000,12372.000000,3.000000
50%,4921.000000,25077.000000,4.000000
75%,7414.000000,38572.000000,5.000000
max,10000.000000,53424.000000,5.000000


In [10]:
s_df = sql_sc.createDataFrame(rating)

In [11]:
s_df.show()

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
|      1|   2077|     4|
|      1|   2487|     4|
|      1|   2900|     5|
|      1|   3662|     4|
|      1|   3922|     5|
|      1|   5379|     5|
|      1|   5461|     3|
|      1|   5885|     5|
|      1|   6630|     5|
|      1|   7563|     3|
|      1|   9246|     1|
|      1|  10140|     4|
|      1|  10146|     5|
|      1|  10246|     4|
|      1|  10335|     4|
+-------+-------+------+
only showing top 20 rows



In [12]:
dfs_train, dfs_test = spark_random_split(s_df, ratio=0.75, seed=42)

In [13]:
RANK = 10
MAX_ITER = 15
REG_PARAM = 0.05

In [48]:
COL_USER = "user_id"
COL_ITEM = "book_id"
COL_RATING = "rating"

In [49]:
als = ALS(
    maxIter=MAX_ITER, 
    rank=RANK,
    regParam=REG_PARAM, 
    userCol=COL_USER, 
    itemCol=COL_ITEM, 
    ratingCol=COL_RATING, 
    coldStartStrategy="drop"
)

model = als.fit(dfs_train)

In [50]:
dfs_pred = model.transform(dfs_test).drop(COL_RATING)

In [51]:
evaluations = SparkRatingEvaluation(
    dfs_test, 
    dfs_pred,
    col_user=COL_USER,
    col_item=COL_ITEM,
    col_rating=COL_RATING,
)

print(
    "RMSE score = {}".format(evaluations.rmse()),
    "MAE score = {}".format(evaluations.mae()),
    "R2 score = {}".format(evaluations.rsquared()),
    "Explained variance score = {}".format(evaluations.exp_var()),
    sep="\n"
)

RMSE score = 0.985685733346917
MAE score = 0.7492083603286372
R2 score = -0.005390252253107741
Explained variance score = 0.025921425256029584


In [ ]:
# Get the cross join of all user-item pairs and score them.
users = dfs_train.select('user_id').distinct()
items = dfs_train.select('book_id').distinct()
user_item = users.crossJoin(items)
dfs_pred = model.transform(user_item)

# Remove seen items.
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    dfs_train.alias("train"),
    (dfs_pred['user_id'] == dfs_train['user_id']) & (dfs_pred['book_id'] == dfs_train['book_id']),
    how='outer'
)

dfs_pred_final = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.rating"].isNull()) \
    .select('pred.' + 'user_id', 'pred.' + 'book_id', 'pred.' + "prediction")

dfs_pred_final.show()

In [52]:
dfs_rec = model.recommendForAllUsers(10)

In [53]:
dfs_rec.show(10)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|    148|[{4653, 4.7775683...|
|    463|[{4090, 5.9260564...|
|    471|[{9486, 3.783837}...|
|    496|[{6443, 6.955799}...|
|    833|[{489, 6.8290014}...|
|   1088|[{2869, 4.9752235...|
|   1238|[{1462, 4.0536976...|
|   1342|[{5449, 5.2474437...|
|   1580|[{4643, 5.306282}...|
|   1591|[{4154, 3.3804708...|
+-------+--------------------+
only showing top 10 rows



In [42]:
users = dfs_train.select(als.getUserCol()).distinct().limit(3) #get 3 distinct random user_id

dfs_rec_subset = model.recommendForUserSubset(users, 10) #return top 10 items for each users

In [43]:
dfs_rec_subset.show(10)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|   8484|[{2236, 6.0364237...|
|   8440|[{146, 5.0934772}...|
|  11945|[{7401, 5.7783537...|
+-------+--------------------+



In [46]:
users = dfs_train.where(dfs_train.user_id == 8484) #get user_id 8484

dfs_rec_subset = model.recommendForUserSubset(users, 10) #return top 10 items for each users

In [47]:
dfs_rec_subset.show(10)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|   8484|[{2236, 6.0364237...|
+-------+--------------------+

